In [ ]:
!pip install torch

     |███████████████▌                | 324.3MB 128.4MB/s eta 0:00:03

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████▎    | 571.6MB 8.8MB/s eta 0:00:12

     |███████████████████████████▌    | 576.7MB 8.8MB/s eta 0:00:11

     |███████████████████████████▊    | 581.7MB 8.8MB/s eta 0:00:11

...

     |████████████████████████▍       | 557.8MB 181.4MB/s eta 0:00:01

     |████████████████████████▌       | 559.9MB 181.4MB/s eta 0:00:01

     |████████████████████████▋       | 562.0MB 181.4MB/s eta 0:00:01

     |████████████████████████▊       | 564.0MB 181.4MB/s eta 0:00:01

     |████████████████████████████████| 731.7MB 190.4MB/s eta 0:00:01
     |████████████████████████████████| 124.2MB 98.8MB/s eta 0:00:01
     |████████████████████████████████| 196.0MB 136.4MB/s eta 0:00:01
     |████████████████████████████████| 14.1MB 96.8MB/s eta 0:00:01
     |████████████████████████████████| 23.7MB 78.7MB/s eta 0:00:01
     |████████████████████████████████| 174kB 75.1MB/s eta 0:00:01
     |████████████████████████████████| 133kB 96.1MB/s eta 0:00:01
     |████████████████████████████████| 542kB 107.9MB/s eta 0:00:01
     |████████████████████████████████| 20.2MB 86.3MB/s eta 0:00:01
ERROR: mosaic-ai-serving 1.0.0 has requirement Flask==2.1.1; python_version >= "3.7", but you'll have flask 2.2.5 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement Jinja2==3.0.3, but you'll have jinja2 3.1.2 which is incompatible.
ERROR: mosaic-ai-serving 1.0.0 has requirement matplotlib==3.6.0; python_version >= "3.8", but you'll have matplotlib 3.7.3 which is i

In [ ]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
